In [1]:
import firebase_admin
from firebase_admin import credentials, firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("admin-key.json")  
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [2]:
import numpy as np
from scipy.signal import find_peaks

def extract_features_from_session(session, timestep=0.05):
    data = session["acceleration"]

    ax = np.array([d['accelerometer']['x'] for d in data])
    ay = np.array([d['accelerometer']['y'] for d in data])
    az = np.array([d['accelerometer']['z'] for d in data])
    mag = np.sqrt(ax**2 + ay**2 + az**2)

    velocity = np.sum(mag) * timestep
    variability = np.std(mag)
    peaks, _ = find_peaks(mag, distance=10)
    frequency = len(peaks) / (len(mag) * timestep)

    return {
        "velocity": float(velocity),
        "variability": float(variability),
        "frequency": float(frequency)
    }

In [3]:
def compare_sessions(pre_session, post_session):
    pre_features = extract_features_from_session(pre_session)
    post_features = extract_features_from_session(post_session)

    print("Comparison:")
    for key in pre_features:
        pre_val = pre_features[key]
        post_val = post_features[key]
        delta = post_val - pre_val
        direction = "↑" if delta > 0 else "↓"
        print(f"    {key}: BEFORE = {pre_val:.3f}, AFTER = {post_val:.3f}, Δ = {delta:.3f} {direction}")

In [ ]:
patient_ids = ["11111", "22222", "33333"]  # Add more as needed
activities = ["walk", "sit", "upstairs"]
locations = ["DominantHand", "DominantWrist", "Non-DominantHand"]  # Location variants

for patient_id in patient_ids:
    print(f"\nPatient ID: {patient_id}")
    
    for activity in activities:
        print(f"\nActivity: {activity}")

        found_any = False

        for location in locations:
            pre_path = db.collection("data").document("BeforeTreatment")\
                .collection(patient_id).document(activity)\
                .collection(location)
            post_path = db.collection("data").document("AfterTreatment")\
                .collection(patient_id).document(activity)\
                .collection(location)

            pre_ids = sorted([doc.id for doc in pre_path.stream()])
            post_ids = sorted([doc.id for doc in post_path.stream()])

            if not pre_ids and not post_ids:
                continue
            if not pre_ids or not post_ids:
                continue

            found_any = True
            pre_doc = pre_path.document(pre_ids[-1]).get()
            post_doc = post_path.document(post_ids[-1]).get()

            if pre_doc.exists and post_doc.exists:
                pre_session = pre_doc.to_dict()
                post_session = post_doc.to_dict()
                print(f"  Location: {location}")
                compare_sessions(pre_session, post_session)
                break  
            else:
                continue

        if not found_any:
            print("  No matching pre/post session found in any location.")



Patient ID: 11111

Activity: walk
  No matching pre/post session found in any location.

Activity: sit
  Location: DominantWrist
Comparison:
    velocity: BEFORE = 157.362, AFTER = 153.033, Δ = -4.328 ↓
    variability: BEFORE = 0.057, AFTER = 0.019, Δ = -0.038 ↓
    frequency: BEFORE = 1.303, AFTER = 1.364, Δ = 0.061 ↑

Activity: upstairs
  Location: DominantWrist
Comparison:
    velocity: BEFORE = 231.855, AFTER = 212.839, Δ = -19.016 ↓
    variability: BEFORE = 0.382, AFTER = 0.381, Δ = -0.001 ↓
    frequency: BEFORE = 1.065, AFTER = 0.912, Δ = -0.153 ↓

Patient ID: 22222

Activity: walk
  Location: DominantWrist
Comparison:
    velocity: BEFORE = 123.927, AFTER = 134.035, Δ = 10.108 ↑
    variability: BEFORE = 0.177, AFTER = 0.211, Δ = 0.034 ↑
    frequency: BEFORE = 1.159, AFTER = 1.039, Δ = -0.120 ↓

Activity: sit
  Location: DominantWrist
Comparison:
    velocity: BEFORE = 116.189, AFTER = 150.367, Δ = 34.178 ↑
    variability: BEFORE = 0.055, AFTER = 0.004, Δ = -0.051 ↓
    fr